In [ ]:
import numpy as np
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch.nn import ModuleList, Embedding
from torch.nn import Sequential, ReLU, Linear
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.utils import degree
from torch_geometric.data import DataLoader
from torch_geometric.nn import PNAConv, BatchNorm, global_add_pool

import time
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
dist_th = 12.0  # threshold distance between CA atoms below which edges will be built
idx_mode = 0  # first mode = 0, last mode = 63

batch_size = 32
epochs = 100

num_layers = 4
patience = 10

np.random.seed(0)
torch.manual_seed(0)

task_name = 'all_th'+str(int(dist_th))+'_mode'+str(idx_mode)

In [ ]:
data_list_raw = torch.load('data/'+task_name+'.pt')

In [ ]:
data_list = [data for data in data_list_raw if data.y < 8]  # remove outlier

N_data = len(data_list)
print(f'Number of proteins: {N_data}')

In [ ]:
idx_val_and_test = np.random.choice(N_data, int(np.floor(0.2 * N_data)), replace=False)
idx_all = list(range(N_data))
idx_train = list(set(idx_all) - set(idx_val_and_test))
idx_val = idx_val_and_test[:int(np.floor(0.5*len(idx_val_and_test)))]
idx_test = idx_val_and_test[int(np.floor(0.5*len(idx_val_and_test))):]

train_dataset = [data_list[i] for i in idx_train]
val_dataset = [data_list[i] for i in idx_val]
test_dataset = [data_list[i] for i in idx_test]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of validation graphs: {len(val_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
mpl.rcParams['axes.linewidth'] = 10
mpl.rcParams['xtick.major.size'] = 30
mpl.rcParams['xtick.major.width'] = 10
mpl.rcParams['ytick.major.size'] = 30
mpl.rcParams['ytick.major.width'] = 10

def plot_freqs_hist(freqs):
    fontsize = 150
    plt.figure(figsize=(50,40))
    plt.gcf().subplots_adjust(left=0.2)
    num_bins = 100
    n, bins, patches = plt.hist(freqs, bins=num_bins)
    plt.xlabel('Frequency (cm$^{-1}$)', fontsize=fontsize)
    plt.ylabel('Count', fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.show()

def show_freqs_info(data_list):
    freqs = torch.zeros(len(data_list), dtype=torch.float)
    for index, data in enumerate(data_list):
        freqs[index] = data.y
    (max_freqs, idx_max_freqs) = torch.max(freqs, 0)
    print(f'Maximum frequency in dataset: {max_freqs:.4f}')
    print(f'Index of maximum frequency in dataset: {idx_max_freqs:.4f}')
    print(f'Mean of frequencies in dataset: {torch.mean(freqs):.4f}')
    print(f'Standard deviation of frequencies in dataset: {torch.std(freqs):.4f}')
    plot_freqs_hist(freqs.numpy())

show_freqs_info(data_list)

In [ ]:
# Compute in-degree histogram over training data.
len_deg_safe = 100  # take a sufficiently large value
deg_safe = torch.zeros(len_deg_safe, dtype=torch.long)
for data in train_dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg_safe += torch.bincount(d, minlength=deg_safe.numel())

for i in range(len_deg_safe):
    if deg_safe[i:].sum() == 0:
        deg = deg_safe[:i]
        break
print(deg)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.node_emb = Embedding(20, 75)

        aggregators = ['mean', 'std']
        scalers = ['identity', 'amplification', 'attenuation']

        self.convs = ModuleList()
        self.batch_norms = ModuleList()
        for _ in range(num_layers):
            conv = PNAConv(in_channels=75, out_channels=75,
                           aggregators=aggregators, scalers=scalers, deg=deg,
                           edge_dim=1, towers=5, pre_layers=1, post_layers=1,
                           divide_input=False)
            self.convs.append(conv)
            self.batch_norms.append(BatchNorm(75))

        self.mlp = Sequential(Linear(75, 50), ReLU(), Linear(50, 25), ReLU(), Linear(25, 1))

    def forward(self, x, edge_index, edge_attr, batch):
        x = self.node_emb(x.squeeze())

        for conv, batch_norm in zip(self.convs, self.batch_norms):
            x = F.relu(batch_norm(conv(x, edge_index, edge_attr.unsqueeze(1))))

        x = global_add_pool(x, batch)
        return self.mlp(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience,
                              min_lr=0.00001)

In [ ]:
def train(epoch):
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = (out.squeeze() - data.y).abs().mean()
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_error = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        total_error += (out.squeeze() - data.y).abs().sum().item()
    return total_error / len(loader.dataset)


loss_and_val_mae = []
for epoch in range(1, epochs+1):
    t = time.time()
    loss = train(epoch)
    val_mae = test(val_loader)
    test_mae = test(test_loader)
    scheduler.step(val_mae)
    loss_and_val_mae.append([loss, val_mae])
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}, Time: {(time.time() - t):.4f}')

In [ ]:
def plot_history(loss_and_val_mae):
    losses_np = np.matrix(loss_and_val_mae)
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Training Loss', color=color)
    ax1.plot(range(epochs), losses_np[:, 0], color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Validation Mean Absolute Error (cm-1)', color=color)
    ax2.plot(range(epochs), losses_np[:, 1], color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    fig.tight_layout()
    plt.show()

plot_history(loss_and_val_mae)

print(f'Final Validation MAE: {loss_and_val_mae[-1][1]:.4f}')

In [ ]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'train_dataset': train_dataset,
            'val_dataset': val_dataset,
            'test_dataset': test_dataset,
            'deg': deg,
            'loss_and_val_mae': loss_and_val_mae,
            'idx_val_and_test': idx_val_and_test,
            }, 'data/'+task_name+'_Nlayer'+str(num_layers)+'_checkpoint'+str(epochs)+'.pt')